In [10]:
from keras import models, layers
from keras.datasets import mnist
import numpy as np
import keras.backend as K

In [11]:
(x_input_train, _), (x_input_test, y_output_test) = mnist.load_data()
x_input_train = x_input_train.astype("float32") / 255.0
x_input_test = x_input_test.astype("float32") / 255.0

In [12]:
latent_dim = 2

# Encoder.
encoder_input = layers.Input(shape=(28, 28))
encoder_output = layers.Reshape((28*28,))(encoder_input) # Flatten
encoder_output = layers.Dense(512, activation="relu")(encoder_output)
encoder = models.Model(encoder_input, encoder_output)

# Decoder.
decoder_input = layers.Input(shape=(latent_dim,))
decoder_output = layers.Dense(512, activation="relu")(decoder_input)
decoder_output = layers.Dense((28*28), activation="sigmoid")(decoder_output)
decoder_output = layers.Reshape((28,28))(decoder_output)
decoder = models.Model(decoder_input, decoder_output)

In [14]:
enc_input = encoder.inputs[0]
enc_output = encoder.outputs[0]
z_mean = layers.Dense(latent_dim, name='z_mean')(enc_output)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(enc_output)
print(z_mean, z_log_var)

Tensor("z_mean_1/BiasAdd:0", shape=(?, 2), dtype=float32) Tensor("z_log_var_1/BiasAdd:0", shape=(?, 2), dtype=float32)


In [15]:
batch = K.shape(z_mean)[0]
dim = K.int_shape(z_mean)[1]
print(batch,dim)

Tensor("strided_slice_2:0", shape=(), dtype=int32) 2


In [16]:
epsilon = K.random_normal(shape=(batch, dim))
epsilon

<tf.Tensor 'random_normal:0' shape=(?, 2) dtype=float32>

In [17]:
ret = z_mean + K.exp(0.5 * z_log_var) * epsilon
ret

<tf.Tensor 'add:0' shape=(?, 2) dtype=float32>

In [ ]:
z =layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])